# Get data from Football-Data.co.uk

**Author:** Iordanou Giannis

**Github:** https://github.com/GiannisIordanou

**Website:** http://pybet.wordpress.com

**Version:** 0.3

**Update date:** 4/6/2015

## Imports

In [1]:
# -*- coding:utf-8 -*-

#Imports

import os
import re
import csv
import numpy as np
import requests
from bs4 import BeautifulSoup
import workerpool
import pandas as pd

## Functions

In [2]:
def get_country_urls(site):
    """
    Get country urls from site.
     
    Parameters
    ----------
    site: string
          Site url
           
    Returns
    -------
    countries: list
               1-D list of strings,
               name of countries
     
    countries_dict: dictionary
                    1-D dictionary containing
                    countries name and urls    
    """
    try:
        headers = {"User-agent": "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) \
                    Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1"}
        r = requests.get(site, headers=headers)
        html = r.text       
        pattern = '<A HREF="(http://www.football-data.co.uk/.*?m.php)"><b>(.*?)</b>'
        matches = list(set(re.findall(pattern, html)))      
        countries_dict = {key:value for (value, key) in matches}
        countries = sorted(countries_dict.keys())
        return countries, countries_dict
    except Exception, e:
        print e
 
 
def get_season_from_csv_url(csv_url):
    """
    Extract season string from csv url
    and return apropriate season.
     
    Parameters
    ----------
    csv_url: url
             csv file url
              
    Returns
    -------
    season: int
            Season, ex. 2015
    """
    try:
        season_string = re.findall("/([0-9]{4})/", csv_url)[0]
        season =  int(season_string[-2:])
        if season > 90:
            cent = "19"
        else:
            cent = "20"
        season = cent + season_string[-2:]
        return season
    except Exception, e:
        print e

        
def get_country_csv_urls(country, countries_dict):
    """
    Get csv urls from a country's page.
     
    Parameters
    ----------
    country: string
             Name of country
              
    countries_dict: dictionary
                    1-D dictionary containing
                    countries name and urls
     
    Returns
    -------
    country_csv_urls: list
              The csv urls of country page
    """
    country_csv_urls = []
    try:
        country_url = countries_dict[country]
        r = requests.get(country_url)
        html = r.text
        soup = BeautifulSoup(html)
        matches = soup.findAll("a")
        for match in matches:
            if "csv" in match["href"]:
                country_csv_url = "".join([site, match["href"]])
                league = match.text
                csv_season = get_season_from_csv_url(match["href"])
                csv_details = [country_csv_url, country, league, csv_season]
                country_csv_urls.append(csv_details)
    except Exception, e:
        print e   
    return country_csv_urls
 

def folder_preparation(files_folder, countries, csv_urls):
    """
    Create the appropriate folders based on
    country names and leagues.
     
    Parameters
    ----------
    files_folder: string
                  Filepath of folder
                  to save files to
     
    countries: list
               List of countries names
     
    csv_urls: list
              1-D list
     
    Returns
    -------
    """
    if not os.path.exists(files_folder):
        try:
            os.mkdir(files_folder)
        except Exception, e:
            print e
             
    for country in countries:
        if country not in os.listdir(files_folder):     
            try:   
                os.mkdir('/'.join([files_folder, country]))
            except Exception, e:
                print e
     
    for country in countries:
        country_leagues = filter(lambda x: x[1] == country, csv_urls)
        for country_league in country_leagues:
            country, league = country_league[1:3]
            league_folder = '/'.join([files_folder, country, league])
            if not os.path.exists(league_folder):
                try:
                    os.mkdir(league_folder)
                except Exception, e:
                    print e


def download_csv_file(csv_info):
    """
    Download csv file.
     
    Parameters
    ----------
    csv_info: list
              1-D list of csv info
     
    Returns
    -------
    """
    csv_url, country, league, season = csv_info
    league_folders = os.listdir('/'.join([files_folder, country]))
    filename = unicode('/'.join([files_folder, country, league, season + '.csv']))
    try:
        r = requests.get(csv_url, stream=True)
        if r.status_code == 200:
            with open(filename, 'wb') as f:
                for chunk in r.iter_content(1024):
                    f.write(chunk)
    except Exception, e:
        print e
        
    
def download_multiple_csv_files(csv_urls, amount):
    """
    Download multiple csv files
     
    Parameters
    ----------
    csv_urls: list
              1-D list
     
    amount: int
            how many files to
            download at once
     
    Returns
    -------
    """
    pool = workerpool.WorkerPool(size=amount)
    pool.map(download_csv_file, csv_urls)
    pool.shutdown()
    pool.wait()

    
def correct_csv(csv_file):
    with open(csv_file, "rb") as f:
        csvfile = csv.reader(f)

        headers = csvfile.next()
        total_headers = len(headers)
        new_lines = [headers]
        for lines in csvfile:
            new_line = lines[:total_headers]
            new_lines.append(new_line)

    with open(csv_file, "wb") as f:
        csvfile = csv.writer(f)
        csvfile.writerows(new_lines)


def process_file(filepath):
    """
    Process file and add to dataframe.
    
    Parameters
    ----------
    filepath: string
              
    
    Returns
    ------- 
    df_file: dataframe
    """
    
    season = int(filepath.split('\\')[-1].split('.')[0])
    country = filepath.split('\\')[-3]
    league = filepath.split('\\')[-2]
    try:
        df_file = pd.read_csv(filepath, sep= ",", na_values=["", " ", "-"])
    except Exception, e:
        correct_csv(filepath) # Lines with more items than expected
        df_file = pd.read_csv(filepath, sep= ",", na_values=["", " ", "-"])
    
    df_file.dropna(axis=0,how='all', inplace=True) # Drop empty lines
    df_file.dropna(axis=1, how="all", inplace=True) # Drop all empty columns
    df_file.dropna(axis=1, thresh = 0.5 * df_file.shape[0], inplace=True) # Drop columns with a few items   
    df_file["Season"] = season
    df_file["Country"] = country
    df_file["League"] = league

    try:
        df_file.rename(columns={"HT": "HomeTeam", "AT": "AwayTeam"}, inplace=True) # Rename HT to HomeTeam and AT to AwayTeam
    except Exception, e:
        print e   
    
    return df_file


def get_ftsc(x):
    """
    Get Full Time Score.
    """
     
    fthg, ftag = x["FTHG"], x["FTAG"]
    if not np.isnan(fthg) and not np.isnan(ftag):
        ftsc = "-".join([str(fthg), str(ftag)]).replace(".0", "")
    else:
        ftsc = "-"
    return ftsc


def get_htsc(x):
    """
    Get Half Time Score.
    """
     
    hthg, htag = x["HTHG"], x["HTAG"]
    if not np.isnan(hthg) and not np.isnan(htag):
        #if hthg.dtype == float and htag.dtype == float:        
        htsc = "-".join([str(hthg), str(htag)]).replace(".0", "")
        #else:
            #htsc = "-"
    else:
        htsc = "-"
    return htsc


def get_ht2sc(x):
    """
    Get second Half Time Score.
    """
     
    ht2hg, ht2ag = x["HT2HG"], x["HT2AG"]
    if not np.isnan(ht2hg) and not np.isnan(ht2ag):
        #if hthg.dtype == float and htag.dtype == float:        
        ht2sc = "-".join([str(ht2hg), str(ht2ag)]).replace(".0", "")
        #else:
            #htsc = "-"
    else:
        ht2sc = "-"
    return ht2sc

def get_date(x):
    """
    Reformat date.
    """
     
    date = x["Date"]
    if date:
        try:
            new_date = datetime.datetime.strftime(datetime.datetime.strptime(date, "%d/%m/%y"), "%d/%m/%Y")
        except:
            new_date = date
    else:
        new_date = "-"
    return new_date    
 

def total_goals_category(x):
    """
    Get total goals category.
    """
     
    total_goals = x
    if not np.isnan(total_goals):
        if 0 <= total_goals < 2:
            total_goals_category = "0-1"
        elif 2 <= total_goals < 4:
            total_goals_category = "2-3"
        elif 4 <= total_goals < 7:
            total_goals_category = "4-6"
        elif 7 <= total_goals:
            total_goals_category = "7"
        else:
            total_goals_category = "-"
    else:
        total_goals_category = "-"
 
    return total_goals_category


def get_hometeam_points(x):
    """
    Get points of hometeam.
    """
    
    result_dict = {"H": 3, "D": 1, "A": 0}
    try:
        hometeam_points = result_dict[x]
    except Exception, e:
        hometeam_points = np.nan
    return hometeam_points 

def get_awayteam_points(x):
    """
    Get points of awayteam.
    """
    result_dict = {"H": 0, "D": 1, "A": 3}
    try:
        awayteam_points = result_dict[x]
    except Exception, e:
        awayteam_points = np.nan
    return awayteam_points
    
def get_ht2r(x):
    """
    Get second Half Time result.
    """
    
    ht2hg, ht2ag = x["HT2HG"], x["HT2AG"]
    if ht2hg > ht2ag:
        ht2r = "H"
    elif ht2hg == ht2ag:
        ht2r = "D"
    elif ht2hg < ht2ag:
        ht2r = "A"
    else:
        ht2r = np.nan
    return ht2r

### Folders

In [3]:
files_folder = "Files"

## Find and download files

In [4]:
site = "http://www.football-data.co.uk/"
countries, countries_dict = get_country_urls(site)
csv_urls = []
for country in countries:
    country_csv_urls = get_country_csv_urls(country, countries_dict)
    csv_urls.extend(country_csv_urls)    
     
folder_preparation(files_folder, countries, csv_urls)
download_multiple_csv_files(csv_urls, 10)

## List csv files

In [5]:
football_files = []
for (dirpath, dirnames, filenames) in os.walk(files_folder):
    football_files.extend(map(lambda x: os.path.join(dirpath, x), filenames))

## Add csv files to dataframe

In [6]:
df_list = []
for each_file in football_files:
    try:
        df_file = process_file(each_file)
        df_list.append(df_file)
    except Exception, e:
        print each_file, e
df = pd.concat(df_list, ignore_index=True)

## Add custom columns

In [7]:
df["HTSC"] = df.apply(get_htsc, axis=1) # Half Time Score
df["FTSC"] = df.apply(get_ftsc, axis=1) # Full Time Score
df["Date"] = df.apply(get_date, axis=1) # Fix Date

df["B365HAOC"] = (df.B365H / df.B365A).apply(lambda x: round(x, 2)) # Bet365 HA Odds Class = B365H / B365A
df["B365HDOC"] = (df.B365H / df.B365D).apply(lambda x: round(x, 2))  # Bet365 HD Odds Class = B365H / B365D
df["B365ADOC"] = (df.B365A / df.B365D).apply(lambda x: round(x, 2))  # Bet365 AD Odds Class = B365A / B365D 
df["FTTG"] = df.FTHG + df.FTAG # Full Time Total Goals
df["FTUO"] = df["FTTG"].apply(lambda x: "O" if x > 2 else "U") # Full Time Under Over 2.5
df["FTTC"] = df.HC + df.AC # Full Time Total Corners
 
df["FTTGC"] = df["FTTG"].apply(total_goals_category) # Full Time Total Goals Category
df["FTGNG"] = df.apply(lambda x: "G" if x["FTHG"] != 0 and x["FTAG"] != 0 else "NG", axis=1) # Full Time Goal No Goal

df["HomeTeam"] = df.apply(lambda x: x["HomeTeam"].strip(), axis=1) # Fix teams with whitespaces
df["AwayTeam"] = df.apply(lambda x: x["AwayTeam"].strip(), axis=1) # Fix teams with whitespaces

In [8]:
df["FTHGD"] = df["FTHG"] - df["FTAG"] # Full Time HomeTeam Goals Difference
df["FTAGD"] = df["FTAG"] - df["FTHG"] # Full Time AwayTeam Goals Difference

In [9]:
df["HTHGD"] = df["HTHG"] - df["HTAG"] # Half Time HomeTeam Goals Difference
df["HTAGD"] = df["HTAG"] - df["HTHG"] # Half Time AwayTeam Goals Difference

In [10]:
df["HT2HG"] = df["FTHG"] - df["HTHG"] # Second Half HomeTeam Goals
df["HT2AG"] = df["FTAG"] - df["HTAG"] # Second Half HomeTeam Goals
df["HT2R"] = df.apply(lambda x: get_ht2r(x), axis=1) # Second Half Result
df["HT2SC"] = df.apply(get_ht2sc, axis=1) # Second Half Score

In [11]:
df["HT2HGD"] = df["HT2HG"] - df["HT2AG"] # Second Half HomeTeam Goals Difference
df["HT2AGD"] = df["HT2AG"] - df["HT2HG"] # Second Half AwayTeam Goals Difference

In [12]:
df["FTHP"] = df["FTR"].apply(lambda x: get_hometeam_points(x)) # Full Time HomeTeam Points
df["FTAP"] = df["FTR"].apply(lambda x: get_awayteam_points(x)) # Full Time AwayTeam Points
df["HTHP"] = df["HTR"].apply(lambda x: get_hometeam_points(x)) # Half Time HomeTeam Points
df["HTAP"] = df["HTR"].apply(lambda x: get_awayteam_points(x)) # Half Time AwayTeam Points
df["HT2HP"] = df["HT2R"].apply(lambda x: get_hometeam_points(x)) # Second Half HomeTeam Points
df["HT2AP"] = df["HT2R"].apply(lambda x: get_awayteam_points(x)) # Second Half AwayTeam Poins

In [13]:
df["HTTG"] = df.HTHG + df.HTAG # Half Time Total Goals
df["HTUO"] = df["HTTG"].apply(lambda x: "O" if x > 2 else "U") # Half Time Under Over 2.5
 
df["HTTGC"] = df["HTTG"].apply(total_goals_category) # Half Time Total Goals Category
df["HTGNG"] = df.apply(lambda x: "G" if x["HTHG"] != 0 and x["HTAG"] != 0 else "NG", axis=1) # Half Time Goal No Goal

df["HT2TG"] = df.HT2HG + df.HT2AG # Second Half Total Goals
df["HT2UO"] = df["HT2TG"].apply(lambda x: "O" if x > 2 else "U") # Second Half Under Over 2.5
 
df["HT2TGC"] = df["HT2TG"].apply(total_goals_category) # Second Half Total Goals Category
df["HT2GNG"] = df.apply(lambda x: "G" if x["HT2HG"] != 0 and x["HT2AG"] != 0 else "NG", axis=1) # Second Half Goal No Goal

## Process dataframe

In [14]:
# Rearrange columns
proper_columns_order = ['Season', 'Country', 'League', 'Div', 'Date',
                        'HomeTeam', 'AwayTeam', 
                        'FTSC', 'FTR', 'FTHP', 'FTAP', 'FTHG', 'FTAG', 'FTHGD', 'FTAGD',
                        'HTSC', 'HTR', 'HTHP', 'HTAP', 'HTHG', 'HTAG', 'HTHGD', 'HTAGD',
                        'HT2SC', 'HT2R', 'HT2HP', 'HT2AP', 'HT2HG', 'HT2AG', 'HT2HGD', 'HT2AGD',
                        'FTUO', 'FTGNG', 'FTTG', 'FTTGC', 
                        'HTUO', 'HTGNG', 'HTTG', 'HTTGC', 
                        'HT2UO', 'HT2GNG', 'HT2TG', 'HT2TGC',
                        'B365HAOC', 'B365HDOC', 'B365ADOC', 
                        'Attendance', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HHW', 'AHW', 
                        'HC', 'AC', 'FTTC', 'HF', 'AF', 'HO', 'AO',  'HY', 'AY', 'HR', 'AR', 'HBP', 
                        'ABP', 'B365H', 'B365D', 'B365A']
 
other_columns = [i for i in df.columns if i not in proper_columns_order]
df = df[proper_columns_order + other_columns]

# Change columns to upper case
df.columns = map(str.upper, df.columns)

# Fill na values
df.fillna("-", inplace=True) 

## Dataframe

In [15]:
df.head()

,SEASON,COUNTRY,LEAGUE,DIV,DATE,HOMETEAM,AWAYTEAM,FTSC,FTR,FTHP,...,SOH,SYA,SYD,SYH,VCA,VCD,VCH,WHA,WHD,WHH
0,1996,Belgium,Jupiler League,B1,04/08/95,Aalst,Anderlecht,3-1,H,3,...,-,-,-,-,-,-,-,-,-,-
1,1996,Belgium,Jupiler League,B1,05/08/95,Cercle Brugge,Antwerp,3-2,H,3,...,-,-,-,-,-,-,-,-,-,-
2,1996,Belgium,Jupiler League,B1,05/08/95,Charleroi,Mechelen,4-2,H,3,...,-,-,-,-,-,-,-,-,-,-
3,1996,Belgium,Jupiler League,B1,05/08/95,Lierse,Harelbeke,0-1,A,0,...,-,-,-,-,-,-,-,-,-,-
4,1996,Belgium,Jupiler League,B1,05/08/95,Molenbeek,Beveren,1-0,H,3,...,-,-,-,-,-,-,-,-,-,-


In [16]:
df.shape

(156692, 137)

## Save to file

In [17]:
df.to_csv(os.path.join(files_folder, "Football-Data_plus.csv"), index=False)
df.to_excel(os.path.join(files_folder, "Football-Data_plus.xlsx"), index=False)
print "Files saved."

Files saved.
